https://www.youtube.com/watch?v=cZdGG_B-S60

In [ ]:
!pip install cohere
!pip install numpy
!pip install annoy
!pip install pandas

In [2]:
import cohere
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from annoy import AnnoyIndex
import numpy as np
import pandas as pd

In [3]:
question = "What are the benefits of using renewable energy sources over fossil fuels?"

text = "Renewable energy is derived from natural processes that are replenished constantly, such as sunlight, wind, rain, tides, waves, and geothermal heat. Unlike fossil fuels, which are finite and emit greenhouse gases contributing to climate change, renewable energy sources offer a sustainable and cleaner alternative. Solar panels convert sunlight directly into electricity, while wind turbines harness the kinetic energy of wind to produce power. Hydroelectric power, generated by the flow of water in rivers or dams, has been a longstanding source of renewable energy. Geothermal energy taps into the Earth's internal heat, and biomass energy comes from organic materials. As technology advances and costs decrease, renewable energy is becoming increasingly viable and essential in reducing our carbon footprint and combating global warming."

In [4]:
texts = text.split("\n\n")

texts = np.array([t.strip(' \n') for t in texts if t])

co = cohere.Client('kktRBqXSFGGkxW4hgxX7qRyYrh530IKTHjmw86cv')

response = co.embed(
    texts = texts.tolist(),
).embeddings

In [5]:
embeds = np.array(response)

search_index = AnnoyIndex(embeds.shape[1], 'angular')

for i in range(len(embeds)):
  search_index.add_item(i, embeds[i])

search_index.build(10)
search_index.save('test.ann')

True

In [6]:
def search_text(query):
  query_embed = co.embed(texts = [query]).embeddings
  similar_item_ids = search_index.get_nns_by_vector(query_embed[0], 10, include_distances=True)
  search_results = texts[similar_item_ids[0]]
  return search_results

In [7]:
results = search_text(question)
print(results[0])

Renewable energy is derived from natural processes that are replenished constantly, such as sunlight, wind, rain, tides, waves, and geothermal heat. Unlike fossil fuels, which are finite and emit greenhouse gases contributing to climate change, renewable energy sources offer a sustainable and cleaner alternative. Solar panels convert sunlight directly into electricity, while wind turbines harness the kinetic energy of wind to produce power. Hydroelectric power, generated by the flow of water in rivers or dams, has been a longstanding source of renewable energy. Geothermal energy taps into the Earth's internal heat, and biomass energy comes from organic materials. As technology advances and costs decrease, renewable energy is becoming increasingly viable and essential in reducing our carbon footprint and combating global warming.


In [10]:
def ask_llm(question, num_generations=1):
  results = search_text(question)
  context = results[0]

  prompt = f"""
  More information on this topic is here cool.com:
  {context}
  Question : {question}

  Extract the answer of the question from the text provided.
  if the text does not contain answer,
  reply that the answer is not available."""

  prediction = co.generate(
      prompt = prompt,
      max_tokens = 70,
      model = "command-nightly",
      temperature = 0.5,
      num_generations = num_generations
  )

  return prediction.generations

In [11]:
results = ask_llm(question,)
print(results[0])

id='dd2ceaaa-647b-4409-89ef-3bbac093a463' text='The benefits of using renewable energy sources over fossil fuels are their sustainability and reduced environmental impact. Unlike finite fossil fuels, renewable sources are constantly replenished and do not contribute to climate change through greenhouse gas emissions. As technology advances, renewable energy is becoming more viable and cost-effective, making it essential in reducing our carbon footprint and mitigating global warming.' index=None likelihood=None token_likelihoods=None finish_reason='COMPLETE'


In [12]:
question = "How do solar panels generate electricity?"
results = ask_llm(question,)
print(results[0])

id='b669193f-b884-4714-96f3-0db36738865f' text='Solar panels convert sunlight directly into electricity.' index=None likelihood=None token_likelihoods=None finish_reason='COMPLETE'


In [15]:
!ls sample_data

anscombe.json		     california_housing_train.csv  mnist_train_small.csv
california_housing_test.csv  mnist_test.csv		   README.md
